Определение языка (language detection)
--------------------

* **Множество случаев** — тексты на разных языках
* **Множество классов** — языки

### Первый метод: частотные слова

Очень простой и неплохой по качеству метод. Сначала создаем частотный словарь для всех языков и берем самые частотные слова. После этого для каждого текста, который нам надо расклассифицировать, смотрим, частотных слов какого языка в нем больше - тот язык и выбираем.

Метод неплохо работает на текстах длиннее 50 слов и быстро имлементируется. 

В качестве корпусов и текстов для тестирования будем использовать статьи Википедии на разных языках. Скачать Википедию можно различными способами:

* Дампы википедии: https://dumps.wikimedia.org/backup-index.html

* wikiextractor: http://medialab.di.unipi.it/wiki/Wikipedia_Extractor

* annotated_wikiextractor: https://github.com/jodaiber/Annotated-WikiExtractor

* wikipedia: https://pypi.python.org/pypi/wikipedia/

#### Скачаем немного википедии для тестов
Воспользуемся пакетом *wikipedia*:

`pip install wikipedia`

In [1]:
def get_texts_for_lang(lang, n=10): # функция для скачивания статей из википедии
    wikipedia.set_lang(lang)
    wiki_content = []
    pages = wikipedia.random(n)
    for page_name in pages:
        try:
            page = wikipedia.page(page_name)
        except wikipedia.exceptions.WikipediaException:
            print('Skipping page {}'.format(page_name))
            continue

        wiki_content.append('{}\n{}'.format(page.title, page.content.replace('==', '')))

    return wiki_content

In [2]:
import wikipedia # скачиваем по 100 статей для каждого языка. Это может занять какое-то время (5-10 минут. как правило)

wiki_texts = {}
for lang in ('kk', 'uk', 'be', 'fr'): # казахский в википедии — это kk,
                                      # украинский — uk, а белорусский — be
    wiki_texts[lang] = get_texts_for_lang(lang, 100)
    print(lang, len(wiki_texts[lang]))

C:\Users\Masha\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\Masha\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


Skipping page Шілік
Skipping page Ихуан
Skipping page Жалаңашкөл
kk 97
Skipping page Сирник
Skipping page Окі
Skipping page Котлярівське
Skipping page Кірай
Skipping page Лаврики
Skipping page Утішне
Skipping page Б'єль
Skipping page Білий крук
Skipping page Соломіївська сільська рада
uk 91
Skipping page Рамановіч
Skipping page Валатоўскі
Skipping page Лукашэнка (значэнні)
Skipping page Церасполле
Skipping page Ізяславічы
Skipping page Будзькаўшчына
Skipping page Данілкі
Skipping page Турэйск
be 92
Skipping page Saviano
Skipping page Élections générales sud-africaines de 2004
Skipping page Pusher
Skipping page AWS
Skipping page Saint-Denis-Catus
Skipping page Sanussi
fr 94


In [3]:
print(wiki_texts['kk'][0]) # распечатаем пару текстов, чтобы убедиться, что все хорошо
print(wiki_texts['fr'][0])

Станлэй (Айдахо)
Станлэй (ағылш. Stanley) — Айдахо штатының Кастер округіне жататын АҚШ қаласы. Қаланың жер аумағы — 1.6 км². FIPS коды — 16-76780.


 Тұрғыны 
2010 жылы қала тұрғыны 100 адамды құрады.


 Дереккөздер 


 Сыртқы cілтемелер 
АҚШ-тың барлық қалалары жайында статистикалар (ағыл.)
U.S. Census Bureau. (ағыл.)
Hyloxalus idiomelus
Hyloxalus idiomelus est une espèce d'amphibiens de la famille des Dendrobatidae.


 Répartition 
Cette espèce est endémique du Pérou. Elle se rencontre dans la province de Bellavista dans la région de San Martín et dans la Ucayali dans la région de Loreto de 1 620 à 2 840 m d'altitude dans la cordillère Centrale.


 Description 
Les mâles mesurent jusqu'à 24,8 mm et les femelles jusqu'à 27,8 mm.


 Publication originale 
Rivero, 1991 : New Colostethus (Amphibia, Dendrobatidae) from South America. Breviora, no 493, p. 1-28 (texte intégral).


 Liens externes 

Référence Amphibian Species of the World : Hyloxalus idiomelus (Rivero, 1991) (en) (consulté

#### Считаем частотный список примерно так:

In [31]:
from string import punctuation, digits
punctuation = set(punctuation + '<>«»-–—…“”\n\t ' + digits)

In [81]:
freqs = dict()
for lang in ('kk', 'uk', 'be', 'fr'):
    freqs[lang] = collections.defaultdict(lambda: 0)

In [82]:
import codecs
import collections
import sys

def tokenize(text):
    return text.split(' ')

for corpus in wiki_texts:
    for article in wiki_texts[corpus]:
        for word in tokenize(article.replace('\n', '').lower()):
            word = word.strip(''.join(punctuation))
            if word != '':
                freqs[corpus][word] += 1

In [83]:
for lang in freqs:
    print(lang)
    for word in sorted(freqs[lang], key=lambda w: freqs[lang][w], reverse=True)[:50]:
        print('{}\t{}'.format(freqs[lang][word], word))

kk
128	су
104	және
94	бойынша
77	өзен
75	коды
70	дереккөздер
65	сыртқы
61	жер
57	сілтемелер
53	орналасқан
51	жылы
48	тұрғындарының
45	мәліметтер
45	гз
42	алып
39	адамды
39	саны
37	аумағы
37	км²
37	құрайды
36	жатқан
35	мен
33	ресей
30	мәліметтері
30	алабы
30	тағы
28	жылғы
28	ресми
27	құрамына
26	халық
26	ұлттық
26	мемлекеттік
26	коммуна
25	жалпы
25	insee
24	қарасты
24	бұл
24	индексі
23	жатады
23	өзенінің
23	қараңыз
23	пошта
22	болып
22	шамасында
22	тығыздығы
21	бөлігі
21	да
21	де
20	ж
20	демографиясы
uk
293	в
267	у
228	на
220	з
162	і
158	та
98	до
87	року
76	за
72	від
70	році
53	для
53	населення
52	а
49	примітки
49	що
47	посилання
46	також
43	км
38	м
37	був
35	й
35	було
34	як
31	не
31	осіб
30	він
29	of
26	про
26	області
25	його
24	the
24	англ
24	під
24	див
23	є
23	с
22	міста
21	при
21	по
21	близько
20	джерела
20	р
20	де
19	або
19	село
18	були
18	району
18	го
18	час
be
482	і
448	у
394	ў
324	на
323	з
141	да
123	па
86	годзе
83	не
73	а
64	што
62	ад
54	быў
53	для
52	спасылкі
51	шахматы
51	год

#### Нужно сделать это для каждого языка и отфильтровать повторяющиеся

Теперь можно загружать готовые частотные словари и классифицировать тексты, просто считая количество слов в каждом.

In [27]:
wiki_texts_test = {}
for lang in ('kk', 'uk', 'be', 'fr'): # казахский в википедии — это kk,
                                      # украинский — uk, а белорусский — be
    wiki_texts_test[lang] = get_texts_for_lang(lang, 100)
    print(lang, len(wiki_texts_test[lang]))

kk 100


C:\Users\Masha\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\Masha\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


Skipping page Подушка (значення)
Skipping page Ардатово
Skipping page Керра
Skipping page Рожков
Skipping page Мрійник (значення)
Skipping page Стецівська сільська рада
Skipping page Куцевич
Skipping page Делегатський провулок
Skipping page Джанелідзе
uk 91
Skipping page Барычэўскі
Skipping page Мак-Мілан
Skipping page Фердынанд IV
Skipping page Даўгалессе
Skipping page Давыдаўка
Skipping page Бярэзніца
Skipping page Браўн
be 93
Skipping page Eschenau
Skipping page I Feel Good
Skipping page Exile
Skipping page SDF
Skipping page Le Triton
Skipping page CMY
fr 94


In [84]:
def define_lang(text):
    
    decision = dict()
    for lang in ('kk', 'uk', 'be', 'fr'):
        decision[lang] = 0
    
    text = text.split()
    for word in text:
        word = word.lower().strip(''.join(punctuation))
        for lang in freqs:
            if word in freqs[lang]:
                decision[lang] += 1
                
    return sorted(decision, key=lambda lang: decision[lang], reverse=True)[0]

In [89]:
real = list()
found = list()
for lang in wiki_texts_test:
    for text in wiki_texts_test[lang]:
        f = define_lang(text)
        found.append(f) 
        real.append(lang)
        if f != lang:
            print(lang, f, text)

kk fr Пало Альто
Пало Альто (Испан: пало: "бұтақ" және альто: "ұзын") Калифорния штатында орналасқан қала. Стэнфорд Университеті және Силикон алқабы осы қалада орналасқан.


 Тарихы 
1769 жылы Гаспар де Портола осы жерде алғаш орнығады.


 Географиясы 


= Топографиясы =


= Қоршаған орта =


= Климаты =


 Жергілікті басқару 


 Саясат 


 Демографиясы 


= 2010 =


= 2000 =


 Тұрғын үй мәселесі 


 Экономикасы 

Amazon.com's A9.com
AOL Inc.
Aricent
Better Place
CPI International
Disney World
Facebook
Hewlett-Packard
IDEO
Mashable
Mopay
Nanosys
Ning
PAIX
Palantir Technologies
Palo Alto Medical Foundation
Palo Alto Research Center (PARC)
Pinterest
Playdom
Socialtext
Space Systems/Loral
Tapulous
Tesla Motors
Tibco Software
Varian Medical Systems
VMware
Wilson Sonsini Goodrich & Rosati
Xerox
MetricStream


 Ерекшеленген мекемелер тізімі : 
Accenture (Бұынғы солтүстік америкалық HQ. Қазіргі Ирландиялық HQ)
BMW (Технология)
CNF Inc.
Dell
Disneyland
EPRI
Genencor (Пало Альтодағы)
Groupon
L

In [86]:
from sklearn.metrics import classification_report

In [87]:
print(classification_report(real, found))

             precision    recall  f1-score   support

         be       1.00      1.00      1.00        93
         fr       0.99      1.00      0.99        94
         kk       1.00      0.99      0.99       100
         uk       1.00      1.00      1.00        91

avg / total       1.00      1.00      1.00       378



### Второй метод: частотные символьные n-граммы

Создадим функцию, которая преобразовывает строку в массив n-грамм заданной длины.

In [46]:
from itertools import islice, tee

def make_ngrams(text):
    N = 3 # задаем длину n-граммы
    ngrams = zip(*(islice(seq, index, None) for index, seq in enumerate(tee(text, N))))
    ngrams = [''.join(x) for x in ngrams]
    return ngrams

Теперь создадим частотные словари n-грамм аналогично первому методу.

In [66]:
freqs_n = dict()
for lang in ('kk', 'uk', 'be', 'fr'):
    freqs_n[lang] = collections.defaultdict(lambda: 0)

In [67]:
for lang in wiki_texts:
    for article in wiki_texts[lang]:
        for ngram in make_ngrams(article.replace('\n', '').lower()):
            freqs_n[lang][ngram] += 1

In [68]:
for lang in freqs_n:
    print(lang)
    for word in sorted(freqs_n[lang], key=lambda w: freqs[lang][w], reverse=True)[:50]:
        print('{}\t{}'.format(freqs_n[lang][word], word))

kk
726	   
385	ың 
380	ала
338	нда
316	ан 
311	ер 
300	ағы
289	ғы 
288	 — 
279	 қа
279	ен 
278	 жа
278	ынд
275	ның
273	асы
262	ары
230	дер
221	ық 
219	 де
216	сын
211	 бо
207	ыны
207	ына
198	ің 
195	ды 
185	 са
185	 ре
184	да 
184	ара
184	сы 
181	алы
176	 ба
171	не 
171	нде
170	ы. 
170	ылы
169	 өз
165	тер
163	 құ
160	нің
159	тар
157	ста
157	ы —
157	 су
156	 жы
156	 ал
155	етт
154	ды.
154	лық
152	ған
uk
503	 на
428	 пр
403	на 
394	ня 
374	 по
365	ння
365	ськ
356	ого
345	го 
342	 ро
341	 за
322	ий 
292	 — 
280	 в 
273	 та
269	 ві
260	 у 
238	енн
229	ере
226	ько
219	та 
216	ні 
213	від
205	 з 
211	их 
203	про
200	ста
202	 ст
204	 до
188	анн
188	ний
185	при
179	ої 
179	 19
183	ів 
171	ван
178	пер
173	 пі
166	 ко
170	льн
167	лен
162	 ви
157	ся 
159	а п
150	ть 
150	 і 
151	аль
148	ка 
149	ом 
144	ку 
be
805	 па
645	 на
592	на 
494	 — 
487	 ка
481	 пр
475	 і 
472	ай 
450	ага
446	кі 
434	 у 
432	га 
404	ны 
396	ста
394	 ў 
382	пра
375	ска
369	ава
366	пер
355	аў 
348	льн
329	дзе
328	ць 
328	 пе

#### Нужно сделать это для каждого языка и отфильтровать повторяющиеся

Теперь, как и в предыдущем методе, можно загружать готовые частотные словари n-грамм и классифицировать тексты, просто подсчитывая частотные n-граммы в каждом.

In [69]:
def define_lang_ngram(text):
    
    decision = dict()
    for lang in ('kk', 'uk', 'be', 'fr'):
        decision[lang] = 0
    
    ngrams = make_ngrams(text)
    for n in ngrams:
        for lang in freqs_n:
            if n in freqs_n[lang]:
                decision[lang] += 1
    
                
    return sorted(decision, key=lambda lang: decision[lang], reverse=True)[0]

In [77]:
real_n = list()
found_n = list()
for lang in wiki_texts_test:
    for text in wiki_texts_test[lang]:
        f = define_lang_ngram(text)
        found_n.append(f) 
        real_n.append(lang)
        if lang != f:
            print(lang, f, text)

kk fr Пало Альто
Пало Альто (Испан: пало: "бұтақ" және альто: "ұзын") Калифорния штатында орналасқан қала. Стэнфорд Университеті және Силикон алқабы осы қалада орналасқан.


 Тарихы 
1769 жылы Гаспар де Портола осы жерде алғаш орнығады.


 Географиясы 


= Топографиясы =


= Қоршаған орта =


= Климаты =


 Жергілікті басқару 


 Саясат 


 Демографиясы 


= 2010 =


= 2000 =


 Тұрғын үй мәселесі 


 Экономикасы 

Amazon.com's A9.com
AOL Inc.
Aricent
Better Place
CPI International
Disney World
Facebook
Hewlett-Packard
IDEO
Mashable
Mopay
Nanosys
Ning
PAIX
Palantir Technologies
Palo Alto Medical Foundation
Palo Alto Research Center (PARC)
Pinterest
Playdom
Socialtext
Space Systems/Loral
Tapulous
Tesla Motors
Tibco Software
Varian Medical Systems
VMware
Wilson Sonsini Goodrich & Rosati
Xerox
MetricStream


 Ерекшеленген мекемелер тізімі : 
Accenture (Бұынғы солтүстік америкалық HQ. Қазіргі Ирландиялық HQ)
BMW (Технология)
CNF Inc.
Dell
Disneyland
EPRI
Genencor (Пало Альтодағы)
Groupon
L

In [78]:
print(classification_report(real_n, found_n))

             precision    recall  f1-score   support

         be       1.00      1.00      1.00        93
         fr       0.99      1.00      0.99        94
         kk       1.00      0.99      0.99       100
         uk       1.00      1.00      1.00        91

avg / total       1.00      1.00      1.00       378



#### Вывод

Оба метода работают одинаково хорошо. Обе модели совершили по одной ошибке, отклассифицировав один казахский текст как текст на французском (выведен выше). В этом текст очень много текста на английском, что и по словам, и по триграммам больше всего пожоже на французский.